In [1]:
import requests
import re 
import time
from bs4 import BeautifulSoup 
from fake_useragent import UserAgent
import pandas as pd
import numpy as np
from random import randint
import selenium.webdriver

In [3]:
city_url = 'https://raw.githubusercontent.com/ucsb-dreamlab/dianpingBusinessScrape/master/big-cities-with-URL%20.csv'
city = pd.read_csv(city_url, index_col= 0).reset_index()
city.head(10)

,City,URL
0,Shanghai,www.dianping.com/shanghai/ch75/g34311
1,Beijing,www.dianping.com/beijing/ch75/g34311
2,Guangzhou,www.dianping.com/guangzhou/ch75/g34311
3,Chengdu,www.dianping.com/chengdu/ch75/g34311
4,Chongqing,www.dianping.com/chongqing/ch75/g34311
5,Shenzhen,www.dianping.com/shenzhen/ch75/g34311
6,Tianjin,www.dianping.com/tianjin/ch75/g34311
7,Wuhan,www.dianping.com/wuhan/ch75/g34311
8,Xi'an,www.dianping.com/xian/ch75/g34311
9,Hangzhou,www.dianping.com/hangzhou/ch75/g34311


In [10]:
from selenium.webdriver import ChromeOptions

1. https://stackoverflow.com/questions/33225947/can-a-website-detect-when-you-are-using-selenium-with-chromedriver
2. https://stackoverflow.com/questions/55501524/how-does-recaptcha-3-know-im-using-selenium-chromedriver/55502835#55502835
3. https://stackoverflow.com/questions/53039551/selenium-webdriver-modifying-navigator-webdriver-flag-to-prevent-selenium-detec



In [11]:
def getHtmlContent(url): 
 UA = UserAgent(verify_ssl = False)  
 #Retrieve cookies with selenium
 options = ChromeOptions()
 #options.add_argument("--disable-blink-features=AutomationControlled")
 options.add_experimental_option("excludeSwitches", ['enable-automation'])
 options.add_experimental_option('useAutomationExtension', False)
 driver = selenium.webdriver.Chrome(chrome_options=options)
 
 
 #driver_option.add_argument("--headless")
 #print(driver.execute_script("return navigator.userAgent;"))
 print("Scraping from "+ url)
 driver.get(url)
 all_cookies = driver.get_cookies()
 cookies_dict = {}
 for cookies in all_cookies:
   cookies_dict[cookies['name']] = cookies['value']
 
 #Set User Agent in request headers
 
 headers = {
    "User-Agent": UA.random
    }
 #Request HTML with session
 try:
   s = requests.Session()
   #r = s.get(url, headers = headers, cookies = {'from-my':'browser'})
   r = s.get(url, headers = headers, cookies= cookies_dict)
   r.raise_for_status()
   if(r.encoding != "UTF-8"):
      r.encoding = r.apparent_encoding
   return r.text
 except:
   return "Request Error"

In [15]:
URL =  "http://"+ city['URL'][2] # add "https" to avoid schema error 
URL

'http://www.dianping.com/guangzhou/ch75/g34311'

In [16]:
HTML = getHtmlContent(URL)

<ipython-input-11-f9d2486382a1>:8: DeprecationWarning: use options instead of chrome_options
  driver = selenium.webdriver.Chrome(chrome_options=options)


Scraping from http://www.dianping.com/guangzhou/ch75/g34311


In [17]:
soup = BeautifulSoup(HTML, "html.parser")


In [18]:
SubURL = []
LibName = []
for div in soup.find_all(class_ = "tit"):
    for a in div.find_all('a', href = True):
        SubURL.append(a.get('href'))
        LibName.append(a.getText().strip())

//*[@id="shop-all-list"]/ul/li[1]/div[2]/div[1]/a/h4

In [19]:
city_lib =pd.DataFrame({'Library_Name':LibName,'URL': SubURL}) 
shop_lib = city_lib[city_lib['URL'].str.contains("shop")].reset_index(drop = True)
shop_lib

,Library_Name,URL
0,伊莱图书馆(天河中心),https://www.dianping.com/shop/k6nI8EcSZwFF8oxn
1,巴黎1区国际艺教连锁(南沙校区),https://www.dianping.com/shop/l212CiA7jkca6DbL
2,丸丸本本(太阳新天地店),https://www.dianping.com/shop/G8jKqpzpyIbnzGO5
3,绘悦·Sky艺术画室,https://www.dianping.com/shop/H3ubGvKratlQOGqS
4,YOUBETTER优贝乐托育·早教(从化校区),https://www.dianping.com/shop/H8dqiPf2NGz3nFpK
5,YOUBETTER优贝乐托育·早教(桥南校区),https://www.dianping.com/shop/l8rHd4iStymPQVr8
6,5+2儿童创意中心乐高机器人(锦麟万博乐高天地校区),https://www.dianping.com/shop/k7j5Mys5C8Iv2vZs
7,海豚悠优阅读早教,https://www.dianping.com/shop/HackwYMYJgKztpUs
8,Dora Play哆啦英式戏剧(东骏广场校区),https://www.dianping.com/shop/k2Dj6rVonZYCp5f0
9,YOUBETTER优贝乐托育·早教(华景新城校区),https://www.dianping.com/shop/l3gA37dW6jzERIQ4


In [20]:
subpage_soup = []
for i in range(0,len(shop_lib)):
    subpage_soup.append(BeautifulSoup(getHtmlContent(url = shop_lib['URL'][i]), "html.parser"))
    time.sleep(randint(1,2))

<ipython-input-11-f9d2486382a1>:8: DeprecationWarning: use options instead of chrome_options
  driver = selenium.webdriver.Chrome(chrome_options=options)


Scraping from https://www.dianping.com/shop/k6nI8EcSZwFF8oxn
Scraping from https://www.dianping.com/shop/l212CiA7jkca6DbL
Scraping from https://www.dianping.com/shop/G8jKqpzpyIbnzGO5
Scraping from https://www.dianping.com/shop/H3ubGvKratlQOGqS
Scraping from https://www.dianping.com/shop/H8dqiPf2NGz3nFpK
Scraping from https://www.dianping.com/shop/l8rHd4iStymPQVr8
Scraping from https://www.dianping.com/shop/k7j5Mys5C8Iv2vZs
Scraping from https://www.dianping.com/shop/HackwYMYJgKztpUs
Scraping from https://www.dianping.com/shop/k2Dj6rVonZYCp5f0
Scraping from https://www.dianping.com/shop/l3gA37dW6jzERIQ4
Scraping from https://www.dianping.com/shop/k1nC59W3ZrhiKnhH
Scraping from https://www.dianping.com/shop/G25rxe56gyVX7w76
Scraping from https://www.dianping.com/shop/k2bHZOcLZGtBSrKh
Scraping from https://www.dianping.com/shop/FvTsXs3fNyN7Xgfa
Scraping from https://www.dianping.com/shop/l4odmwd9zbESOXTA


In [27]:
#subpage_soup[7]

In [51]:
AddressList = []
HourList = []
PhoneNumList = []


In [52]:
for soup in subpage_soup:
    div = soup.find_all(class_ = "address")
    AddressList.append(div[0].find_all(text=True, recursive=False)[1].strip())

In [53]:
AddressList

['东泰社区鸿福东路1号民盈国贸城5号楼',
 '万达广场万达百货',
 '莞太路旺南奥特莱斯3楼303室',
 '南城区西平联景大厦3楼',
 '世博广场D区2楼',
 '香园路33号摩天艺术中心2楼',
 '东纵大道8号3楼',
 '东莞市南城区第一国际财富中心博学城二层',
 '凤岗镇昌盛南路35号四楼',
 '体育路都汇大厦602',
 '未来世界花园169号',
 '莲峰北路沃多夫风临公馆3楼',
 '龙城一路汇星商业中心5栋1606室',
 '东昇路石井综合楼2号楼3楼',
 '东纵路莞城段2号3004室']

In [54]:
for soup in subpage_soup:
    hour_tag = soup.find(class_ = "mod shop-info")
    if hour_tag is not None:
       for li in hour_tag.find_all('li'):
         for span in li.find_all('span'): 
            match = span.text 
            if match == "营业时间":
                str = li.find_all(text=True, recursive=False)[1].strip().replace("\n", " ")
                HourList.append(str) 
    else:
         HourList.append(np.NaN)


In [55]:
HourList

['周一至周日 10:00-20:00',
 '周一至周日 10:00-20:00',
 '周一,周三至周日 09:00-18:00',
 '周一至周日 08:00-18:00',
 '周一至周日 08:00-20:00',
 '周一至周日 08:30-12:00 14:30-18:30',
 '周一至周日 10:00-22:00',
 '周六,周日 08:30-18:30  周三至周五 09:00-21:00',
 '周二至周五 14:00-20:00  周六,周日 10:00-18:30',
 '周六,周日 08:00-21:00  周一至周五 14:00-21:00',
 '周二至周日 09:30-18:00',
 '周一至周日 08:00-18:00',
 '周一至周日 10:00-22:00',
 '周二至周日 09:00-21:00',
 '周一至周日 10:00-22:00']

In [56]:
for soup in subpage_soup: 
    phone_tag = soup.find(class_ = "phone")
    if phone_tag is not None:
        phonenum = phone_tag.find('span')
        PhoneNumList.append(phonenum['data-phone'])
    else: 
        PhoneNumList.append(np.NaN)

In [57]:
PhoneNumList

['15907552646',
 '15907552646',
 '4000303096-04603',
 '4000303096-87514',
 '4000303096-26351',
 '4000303096-27917',
 '4000303096-00006',
 '18929227556',
 '15820888755',
 '4000303096-36220',
 '13377700884',
 '16507690768',
 '0769-81823888',
 '13922931039',
 '0769-21661849']

In [58]:
shop_lib['Address'] = AddressList
shop_lib['Business_Hour'] = HourList
shop_lib['Phone_Number'] = PhoneNumList
shop_lib

,Library_Name,URL,Address,Business_Hour,Phone_Number
0,博沃思·专注力·心理咨询·家庭教育(东城中心),https://www.dianping.com/shop/l9aKfsT5ai99979f,东泰社区鸿福东路1号民盈国贸城5号楼,周一至周日 10:00-20:00,15907552646
1,博沃思注意力心理咨询(南城民盈国贸校区),https://www.dianping.com/shop/l3rGpmYP8Pmi2Y7c,万达广场万达百货,周一至周日 10:00-20:00,15907552646
2,巧虎KIDS成长中心(南城旺南奥特莱斯校区),https://www.dianping.com/shop/kafJm0OSb07Rb0QP,莞太路旺南奥特莱斯3楼303室,"周一,周三至周日 09:00-18:00",4000303096-04603
3,棒吉拉国际早教(东城世博校区),https://www.dianping.com/shop/k8O23jycCAlNs1BD,南城区西平联景大厦3楼,周一至周日 08:00-18:00,4000303096-87514
4,棒吉拉国际早教(南城联景校区),https://www.dianping.com/shop/k2cncDNeqiZbMhzy,世博广场D区2楼,周一至周日 08:00-20:00,4000303096-26351
5,棒吉拉国际早教中心(金域华府店),https://www.dianping.com/shop/l36ITBGXABzNElMs,香园路33号摩天艺术中心2楼,周一至周日 08:30-12:00 14:30-18:30,4000303096-27917
6,上语书田(莞城店),https://www.dianping.com/shop/G7gsOAQ43tvVJmtC,东纵大道8号3楼,周一至周日 10:00-22:00,4000303096-00006
7,书鲸阅读海洋第一国际校区,https://www.dianping.com/shop/i7Ylg8PjQrAWZeR1,东莞市南城区第一国际财富中心博学城二层,"周六,周日 08:30-18:30 周三至周五 09:00-21:00",18929227556
8,橘子绘本馆,https://www.dianping.com/shop/kaue1mlrFmTq7Nas,凤岗镇昌盛南路35号四楼,"周二至周五 14:00-20:00 周六,周日 10:00-18:30",15820888755
9,海洋国际双语阅读馆,https://www.dianping.com/shop/H88gFr4Ts770cgQu,体育路都汇大厦602,"周六,周日 08:00-21:00 周一至周五 14:00-21:00",4000303096-36220


In [76]:
#Summarized the above code into a function
def parse_info_citylib(URL):
    HTML = getHtmlContent(URL)
    soup = BeautifulSoup(HTML, "html.parser")
    
    SubURL = []
    LibName = []
    for div in soup.find_all(class_ = "tit"):
        for a in div.find_all('a', href = True):
            SubURL.append(a.get('href'))
            LibName.append(a.getText().strip())
    
    city_lib =pd.DataFrame({'Library_Name':LibName,'URL': SubURL}) 
    shop_lib = city_lib[city_lib['URL'].str.contains("shop")].reset_index(drop = True)




    subpage_soup = []
    for i in range(0,len(shop_lib)): #Need to change this part
        subpage_soup.append(BeautifulSoup(getHtmlContent(url = shop_lib['URL'][i]), "html.parser"))
        time.sleep(randint(1,2))

    AddressList = []
    HourList = []
    PhoneNumList = []

    for soup in subpage_soup:
        div = soup.find_all(class_ = "address")
        AddressList.append(div[0].find_all(text=True, recursive=False)[1].strip())
    
    for soup in subpage_soup:
        hour_tag = soup.find(class_ = "mod shop-info")
        if hour_tag is not None:
            for li in hour_tag.find_all('li'):
                for span in li.find_all('span'): 
                    match = span.text 
                    if match == "营业时间":
                        str = li.find_all(text=True, recursive=False)[1].strip().replace("\n", " ")
                        HourList.append(str) 
        else:
            HourList.append(np.NaN)
    
    for soup in subpage_soup:
        phone_tag = soup.find(class_ = "phone")
        if phone_tag is not None:
            phonenum = phone_tag.find('span')
            PhoneNumList.append(phonenum['data-phone'])
        else: 
            PhoneNumList.append(np.NaN)
            
    shop_lib['Address'] = AddressList
    shop_lib['Business_Hour'] = HourList
    shop_lib['Phone_Number'] = PhoneNumList


    return shop_lib

In [77]:
Second_URL="http://"+ city['URL'][1]
Second_URL

'http://www.dianping.com/anshan/ch75/g34311'

In [78]:
lib_info = parse_info_citylib(Second_URL)

Scraping from http://www.dianping.com/anshan/ch75/g34311
Scraping from https://www.dianping.com/shop/G7NqN7b6XuDR1Av9
Scraping from https://www.dianping.com/shop/jKWN2u1yfFov84SD
Scraping from https://www.dianping.com/shop/l7uvWW0qRkpJajex
Scraping from https://www.dianping.com/shop/H5t9xip4GPLuIlNt
Scraping from https://www.dianping.com/shop/G64Zmx3FjlgEZzJp
Scraping from https://www.dianping.com/shop/jVQ42SQoGAhH3HC4
Scraping from https://www.dianping.com/shop/H2hW0bGhmUci0kDT


In [79]:
lib_info

,Library_Name,URL,Address,Business_Hour,Phone_Number
0,盛文绘本馆,https://www.dianping.com/shop/G7NqN7b6XuDR1Av9,二一九路26号,NaN,NaN
1,花瓣绘本馆,https://www.dianping.com/shop/jKWN2u1yfFov84SD,自由街劳动大厦北50米,周二至周日 10:00-13:00 13:30-18:30,15998001940
2,啄木鸟,https://www.dianping.com/shop/l7uvWW0qRkpJajex,景子街华润万家超市负一层,周一至周日 09:00-21:00,NaN
3,兰可琪绘本美学生活馆,https://www.dianping.com/shop/H5t9xip4GPLuIlNt,工人街30栋向阳小学东30米,周一至周日 10:00-22:00,15604229606
4,不点的小书库(鞍山大悦城店),https://www.dianping.com/shop/G64Zmx3FjlgEZzJp,胜利北路302号鞍山大悦城F4层,NaN,13609819477
5,啄木鸟,https://www.dianping.com/shop/jVQ42SQoGAhH3HC4,新昌大街西50米,周一至周日 09:00-21:00,15941245428
6,布朗熊亲子绘本馆,https://www.dianping.com/shop/H2hW0bGhmUci0kDT,新东三街与新东二街交叉口东北260米,周一至周日 09:00-18:00,13566634575


In [ ]:
from pathlib import Path  
for i in range(0, len(city)):
    URL = "http://"+ city['URL'][i]
    lib_info = parse_info_citylib(URL)
    filepath = Path('/Users/jma/Documents/DianPing Webscraping')  
    filepath.parent.mkdir(parents=True, exist_ok=True) 
    filename = city['city_name'][i] + '.csv'
    lib_info.to_csv(filename, index = False, encoding='utf_8_sig')